# Time Series Components

## Overview

<style>
    table.custom-table {
        max-width: 1000px;
        width: 100%;
        margin: 0 auto; /* centers the table on the page */
    }

    table.custom-table td {
        background-color: #fff;
    }
    
    table.custom-table th, table.custom-table td {
        text-align: center;
        vertical-align: middle;
        padding: 5px;
        width: 333px; /* distribute the total width equally among three columns */
    }

    table.custom-table img {
        width: 100%;
        display: block; /* removes any gap under the image */
    }
</style>


<table class="custom-table">
    <thead>
        <tr>
            <th>Output: Accumulated Generation</th>
            <th>Output: Technology Relationship</th>
            <th>Input: Hourly Energy Generation</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>
                <img src="src/01_output_2.png" alt="Solar Image 1">
            </td>
            <td>
                <img src="src/01_output_1.png" alt="Solar Image 2">
            </td>
            <td>
                <img src="src/01_input.png" alt="Solar Image 3">
            </td>
        </tr>
    </tbody>
</table>

## Data

In [ ]:
#!

import pandas as pd

df_passenger = pd.read_csv('../data/airline-passengers.csv', parse_dates=['Month'], index_col='Month')
df_passenger = df_passenger.asfreq('MS')
df_passenger

,Passengers
Month,
1949-01-01,112
1949-02-01,118
...,...
1960-11-01,390
1960-12-01,432


## Individual Components

- Trend (T)
- Seasonality (S)
- Residual or Irregular Component (I)

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,63.830808,-42.622475
1949-08-01,148,127.250000,62.823232,-42.073232
...,...,...,...,...
1960-05-01,472,472.750000,-4.506313,3.756313
1960-06-01,535,475.041667,35.402778,24.555556


,Passengers,trend,seasonal,residual,component_sum
Month,,,,,
1949-07-01,148,126.791667,63.830808,-42.622475,148.0
1949-08-01,148,127.250000,62.823232,-42.073232,148.0
...,...,...,...,...,...
1960-05-01,472,472.750000,-4.506313,3.756313,472.0
1960-06-01,535,475.041667,35.402778,24.555556,535.0


## Decomposition by Model

- Additive Model $y_t = T_t + S_t + e_t$
- Multiplicative Model $y_t = T_t \times S_t \times e_t$

In [ ]:
import statsmodels.api as sm #!

data = df_passenger['Passengers'].values
result = sm.tsa.seasonal_decompose(data, model='multiplicative', period=12)

df_component = (df_passenger
 .assign(
    trend = result.trend,
    seasonal = result.seasonal,
    residual = result.resid)
 .dropna())

df_component

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,1.226556,0.951664
1949-08-01,148,127.250000,1.219911,0.953401
...,...,...,...,...
1960-05-01,472,472.750000,0.981378,1.017359
1960-06-01,535,475.041667,1.112776,1.012079


In [ ]:
df_component_mul = df_component.copy()
df_component_mul['component_mul'] = df_component.trend * df_component.seasonal * df_component.residual
df_component_mul #!

,Passengers,trend,seasonal,residual,component_mul
Month,,,,,
1949-07-01,148,126.791667,1.226556,0.951664,148.0
1949-08-01,148,127.250000,1.219911,0.953401,148.0
...,...,...,...,...,...
1960-05-01,472,472.750000,0.981378,1.017359,472.0
1960-06-01,535,475.041667,1.112776,1.012079,535.0


,Passengers,trend,seasonal,residual,component_sum
Month,,,,,
1949-07-01,148,126.791667,63.830808,-42.622475,148.0
1949-08-01,148,127.250000,62.823232,-42.073232,148.0
...,...,...,...,...,...
1960-05-01,472,472.750000,-4.506313,3.756313,472.0
1960-06-01,535,475.041667,35.402778,24.555556,535.0
